In [ ]:
import numpy as np
import imutils
import time
import cv2
from imutils.video import FileVideoStream
from imutils.video import FPS

# Пути к файлам
protoPath = r"C:\Users\Roadmarshal\Downloads\MobileNetSSD_deploy.prototxt.txt"
modelPath = r"C:\Users\Roadmarshal\Downloads\MobileNetSSD_deploy.caffemodel"
confidenceThreshold = 0.2

# Классификации и цвета
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

print("[INFO] Loading model...")
net = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# Путь к видео
video_path = r"C:\Users\Roadmarshal\Downloads\Video Example.mp4"
print("[INFO] Starting video stream...")
fvs = FileVideoStream(video_path).start()
time.sleep(1.0)
fps = FPS().start()
timestamp = time.time()

cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Frame", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while fvs.more():
    frame = fvs.read()
    if frame is None:
        break

    frame = imutils.resize(frame, width=800)
    frame_center = (frame.shape[1] // 2, frame.shape[0] // 2)
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    cv2.line(frame, (frame_center[0], 0), (frame_center[0], h), (0, 255, 0), 2)  # Ось Y
    cv2.line(frame, (0, frame_center[1]), (w, frame_center[1]), (0, 255, 0), 2)  # Ось X

    for i in np.arange(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > confidenceThreshold:
            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            if (endX - startX) < (w * 0.9):
                center_deviation = ((startX + endX) // 2 - frame_center[0], (startY + endY) // 2 - frame_center[1])
                estimated_distance = (7.8 * w) / (endX - startX) if (endX - startX) != 0 else float('inf')

                label = f"Distance: {estimated_distance:.2f} m, " \
                        f"X-offset: {center_deviation[0]}, " \
                        f"Y-offset: {center_deviation[1]}"
                cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[idx], 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 1)

    fps.update()
    current_fps = 1 / (time.time() - timestamp)
    timestamp = time.time()
    cv2.putText(frame, f"FPS: {current_fps:.2f}", (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

fps.stop()
print("[INFO] Approx. FPS: {:.2f}".format(fps.fps()))
cv2.destroyAllWindows()
fvs.stop()